##IMDB MOVIE REVIEW ANALYSIS - CASE STUDY

Review 2 - Implementing Data Mining Algorithms

### GROUP - 3
####Team Members 

Avantika Balaji - CB.EN.U4CSE20309

Kowshikraja T R - CB.EN.U4CSE20331

Sanjay Balaji P - CB.EN.U4CSE20354

Archana Ganapathy - CB.EN.U4CSE20407

Gokul S - CB.EN.U4CSE20420




1. TF - IDF

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate the TfidfVectorizer object
tfidf = TfidfVectorizer()
import nltk
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
data = pd.read_csv('LSH.csv', encoding='latin1')

In [3]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text into words
    words = text.split()

    # Remove stopwords and lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]

    # Join the words back into a string
    text = ' '.join(words)
    return text

# Preprocess the 'review_comment' column to replace NaN values with empty strings
data['review_comment'] = data['review_comment'].fillna('')

# Apply preprocessing to the 'review_comment' column
data['review_comment'] = data['review_comment'].apply(preprocess_text)


In [4]:
# Compute the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(data['review_comment'])

# Get the feature names (words) in the TF-IDF matrix
feature_names = tfidf.get_feature_names_out()


In [5]:
# Get the TF-IDF scores for a specific movie (e.g., movie index 0)
movie_index = 1
tfidf_scores = tfidf_matrix[movie_index].toarray().flatten()

# Sort the feature names (words) based on their TF-IDF scores
sorted_words = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)

# Print the top 10 words with the highest TF-IDF scores for the first movie
top_words = sorted_words[:10]
for word, score in top_words:
    print(f"Word: {word}, TF-IDF Score: {score}")

Word: cinema, TF-IDF Score: 0.23374266185612572
Word: absorbing, TF-IDF Score: 0.1767736199495215
Word: basement, TF-IDF Score: 0.1767736199495215
Word: brutal, TF-IDF Score: 0.1767736199495215
Word: carefully, TF-IDF Score: 0.1767736199495215
Word: choose, TF-IDF Score: 0.1767736199495215
Word: enter, TF-IDF Score: 0.1767736199495215
Word: grace, TF-IDF Score: 0.1767736199495215
Word: leave, TF-IDF Score: 0.1767736199495215
Word: lock, TF-IDF Score: 0.1767736199495215


**INFERENCE:**

Based on the TF-IDF analysis of the movie reviews, we can infer the following:

1. The word "blew" has the highest TF-IDF score of 0.4499, indicating that it is a significant and unique word in the first movie's review. It suggests that the movie had a strong impact on the reviewer.

2. The word "mindbending" has a high TF-IDF score of 0.4201, indicating that it is an important word in the review. It suggests that the movie has a complex and thought-provoking storyline.

3. The word "intricate" has a TF-IDF score of 0.3621, suggesting that the movie's plot or narrative is detailed and involved.

4. The word "visuals" has a TF-IDF score of 0.3361, indicating that the movie's visual aspects, such as special effects or cinematography, are noteworthy.

5. The word "mind" has a TF-IDF score of 0.3153, suggesting that the movie is thought-provoking and engages the audience's intellect.

6. The word "true" has a TF-IDF score of 0.3063, indicating that the movie is perceived as genuine or authentic.

7. The word "plot" has a TF-IDF score of 0.2980, suggesting that the movie's storyline is an important aspect of the review.

8. The word "masterpiece" has a TF-IDF score of 0.2587, indicating that the reviewer considers the movie to be an exceptional piece of work.

9. The word "movie" has a TF-IDF score of 0.1672, suggesting that it is a common word in the reviews and may not provide unique information about the movie.

10. The word "10" has a TF-IDF score of 0.0, indicating that it is not present in the first movie's review or it has very low frequency in the corpus.

From this analysis, we can gather insights about the key aspects and sentiments expressed in the reviews of the first movie. Words with higher TF-IDF scores provide more weight and importance in the context of the specific movie review.

2. LSH

In [6]:

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
import re
re.compile('<title>(.*)</title>')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


re.compile(r'<title>(.*)</title>', re.UNICODE)

In [7]:
import pandas as pd
import numpy as np
import binascii
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from sklearn.metrics import jaccard_score

# Step 1: Shingling
def shingling(text, k):
    shingles = set()
    words = word_tokenize(text)
    for i in range(len(words) - k + 1):
        shingle = " ".join(words[i:i+k])
        shingles.add(str(binascii.crc32(shingle.encode('utf-8'))))  # Convert to string
    return shingles

# Step 2: Min-hashing
def minhashing(shingles, num_hashes):
    hashes = np.inf * np.ones(num_hashes)
    for shingle in shingles:
        hashes = np.minimum(hashes, hash_functions(shingle, num_hashes))
    return hashes

def hash_functions(shingle, num_hashes):
    np.random.seed(42)  # For reproducibility
    a = np.random.randint(1, 1_000_000, size=num_hashes)
    b = np.random.randint(1, 1_000_000, size=num_hashes)
    return (a * int(shingle) + b) % 1_000_000

# Step 3: LSH
def lsh(signature_matrix, bands, rows):
    num_docs = signature_matrix.shape[1]
    buckets = {}

    for b in range(bands):
        bucket_dict = {}
        for doc in range(num_docs):
            band = tuple(signature_matrix[b * rows: (b + 1) * rows, doc])
            if band in bucket_dict:
                bucket_dict[band].append(doc)
            else:
                bucket_dict[band] = [doc]

        for key, docs in bucket_dict.items():
            if len(docs) > 1:
                for i in range(len(docs)):
                    for j in range(i + 1, len(docs)):
                        pair = tuple(sorted([docs[i], docs[j]]))
                        if pair in buckets:
                            buckets[pair] += 1
                        else:
                            buckets[pair] = 1

    return buckets

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union
    return similarity

def find_similar_reviews(reviews, k=5, num_hashes=100, bands=20, rows=5):
    stop_words = set(['that', 'This', 'it', 'is', 'and', 'The', 'of', 'a', 'this', 'A', 'That', 'i','in','the','I','a', 'an', 'the', 
                  'this', 'that', 'these', 'those', 'is', 'are', 'was', 'were', 'am', 'be', 'being', 'been', 'has', 'have', 'had', 
                  'do', 'does', 'did', 'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must', 'of', 'off', 
                  'on', 'in', 'into', 'at', 'to', 'for', 'from', 'with', 'without', 'about', 'above', 'below', 'under', 'over',
                  'through', 'between', 'among','out','of','helpful?', 'helpful.', 'vote.', 
                  'review', 'found', 'Was', 'Permalink', 'out', 'Sign',',','.','movie','my','it.','by','movie,'])

    # Step 1: Shingling
    shingles = []
    for review in reviews:
        if isinstance(review, str):
            review = review.lower()  # Convert to lowercase
            words = word_tokenize(review)  # Tokenize into words
            words = [word for word in words if word not in stop_words]  # Remove specific words
            words = [word for word in words if word.isalpha()]  # Remove non-alphabetic tokens
            shingles.append(" ".join(words))
        else:
            shingles.append("")  # Handle missing or non-string values

    # Print shingle words for first 20 reviews
    print("1. SHINGLING")
    print("\nShingles for 20 reviews: (sample)")
    for i in range(20):
        print(f"Review {i + 1}: {shingles[i]}")

    # Step 2: Min-hashing
    vectorizer = HashingVectorizer(n_features=num_hashes, alternate_sign=False)
    signature_matrix = vectorizer.fit_transform(shingles).toarray()

    # Print signature matrix
    print("\n2. SIGNATURE MATRIX:")
    print()
    print(signature_matrix)
    print()

    # Calculate maximum, minimum, and average of the signature matrix
    max_val = np.max(signature_matrix)
    min_val = np.min(signature_matrix)
    avg_val = np.mean(signature_matrix)

    print("Max value in the signature matrix:", max_val)
    print("Min value in the signature matrix:", min_val)
    print("Average value of the signature matrix:", avg_val)
    print()

    # Step 3: LSH
    similar_pairs = lsh(signature_matrix.T, bands, rows)

    return similar_pairs


# Read the first 8000 lines from the dataset
df = pd.read_csv("LSH.csv")
df.dropna(subset=["review_comment"], inplace=True)
df.drop_duplicates(subset=["review_comment"], inplace=True)
df["review_comment"] = df["review_comment"].apply(lambda x: re.sub(r'\d+', '', str(x)))

reviews = df["review_comment"].tolist()

# Find similar reviews
similar_pairs = find_similar_reviews(reviews)

# Print all similar review pairs
count = 0
printed_pairs = set()
stop_words = set(['that', 'This', 'it', 'is', 'and', 'The', 'of', 'a', 'this', 'A', 'That', 'i','in','the','I','a', 'an', 'the', 
                  'this', 'that', 'these', 'those', 'is', 'are', 'was', 'were', 'am', 'be', 'being', 'been', 'has', 'have', 'had', 
                  'do', 'does', 'did', 'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must', 'of', 'off', 
                  'on', 'in', 'into', 'at', 'to', 'for', 'from', 'with', 'without', 'about', 'above', 'below', 'under', 'over',
                  'through', 'between', 'among','out','of','helpful?', 'helpful.', 'vote.', 'review', 'found', 'Was', 'Permalink',
                  'out', 'Sign',',','.','movie','my','it.','by','movie,'])

max_similarity = 0
max_pair = None
print("\n 3. LSH - LOCALITY SENSITIVE HASHING")


for pair, count in sorted(similar_pairs.items(), key=lambda x: x[1], reverse=True):
    if count > 1 and pair not in printed_pairs:
        review_1 = reviews[pair[0]]
        review_2 = reviews[pair[1]]
        printed_pairs.add(pair)

        words_1 = set(review_1.split())
        words_2 = set(review_2.split())

        similar_words = words_1.intersection(words_2)
        similar_words = similar_words - stop_words  # Remove stop words

        if similar_words:
            similarity = jaccard_similarity(words_1, words_2)

            if similarity > max_similarity:
                max_similarity = similarity
                max_pair = (review_1, review_2)

            print(f"Similar pair: {pair}, count: {count}")
            print(f"Review 1: {review_1}")
            print(f"Review 2: {review_2}")
            print("Similar Words:")
            print(similar_words)
            similarity = jaccard_similarity(words_1, words_2)
            print("Jaccard Similarity:", similarity)
            print()
            print()

print("Pair with the highest Jaccard similarity:")
print(f"Review 1: {max_pair[0]}")
print(f"Review 2: {max_pair[1]}")
print(f"Jaccard Similarity: {max_similarity}")


1. SHINGLING

Shingles for 20 reviews: (sample)
Review 1: intricate plot visuals blew mind true masterpiece
Review 2: you only get watch first time once so choose your state mind carefully film movies dreams reality what sort life best find when you leave cinema return whatever you left enter spectacular brutal enigmatic disturbing beautiful absorbing one favourite characters ever grace screen see often case not as good as like remember secret lock away safe basement somewhere there perfect world us all some perhaps cinema watching
Review 3: when first watch just shocked twist not gon na mention details because ca share any spoilers simply makes complex story makes you question so many things why he keeping her his dreams or whatever your question answering some well written line scene key whole line cobb mal answers all your questions once why like so much some movies potential making complex story someone answer only one sentence you waiting line whole content but most contents mean 

3. BLOOM FILTER

In [12]:
!pip install mmh3
!pip install bitarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 6.2 MB/s eta 0:00:00


In [13]:
import mmh3
from bitarray import bitarray
import csv
import operator

class BloomFilter:
    def __init__(self, size, num_hash_functions):
        self.size = size
        self.num_hash_functions = num_hash_functions
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)

    def add(self, item):
        for seed in range(self.num_hash_functions):
            index = mmh3.hash(item, seed) % self.size
            self.bit_array[index] = 1

    def contains(self, item):
        for seed in range(self.num_hash_functions):
            index = mmh3.hash(item, seed) % self.size
            if not self.bit_array[index]:
                return False
        return True

    def __len__(self):
        return self.size

filter_size = 100000  # Size of the Bloom filter bit array
num_hash_functions = 3  # Number of hash functions to use

bloom_filter = BloomFilter(filter_size, num_hash_functions)
movie_ratings = {}  # Dictionary to store movie ratings and counts

# Load movie review data from a dataset
dataset_path = "Final.csv"

# Assuming the dataset has columns "review_movie_title" and "review_rating"
with open(dataset_path, "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        movie_title = row["review_movie_title"]

        # Skip if movie_title is empty
        if movie_title == '':
            continue

        review_rating_str = row["review_rating"]

        bloom_filter.add(movie_title)

        # Skip empty ratings
        if review_rating_str == '':
            continue

        review_rating = float(review_rating_str)

        # Calculate average rating for each movie
        if movie_title in movie_ratings:
            total_rating, count = movie_ratings[movie_title]
            total_rating += review_rating
            count += 1
            movie_ratings[movie_title] = (total_rating, count)
        else:
            movie_ratings[movie_title] = (review_rating, 1)

# Calculate average ratings for movies
average_ratings = {}
for movie_title, (total_rating, count) in movie_ratings.items():
    average_ratings[movie_title] = total_rating / count

# Sort movies based on average rating in descending order
sorted_movies = sorted(average_ratings.items(), key=operator.itemgetter(1), reverse=True)

# Display the top ten movies
print("Top Ten Movies:")
for i, (movie, avg_rating) in enumerate(sorted_movies[:10]):
    print(f"{i+1}. {movie} - Average Rating: {avg_rating:.2f}")


Top Ten Movies:
1. The Matrix - Average Rating: 10.00
2. Interstellar - Average Rating: 9.62
3. The Godfather - Average Rating: 9.33
4. The Shawshank Redemption - Average Rating: 9.17
5. Inception - Average Rating: 5.35


4. COLLABORATIVE FILTERING

In [14]:
import numpy as np
import pandas as pd
# df=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/Final.csv")
# df

In [15]:
def fill(movie_title,movie_username,movie_rating,df,what_to_fill):
  df[movie_title]=df[movie_title].fillna(what_to_fill)
  df[movie_username]=df[movie_username].dropna()
  mean_value=np.round(shaw[movie_rating].mean())
  df[movie_rating].fillna(value=mean_value, inplace=True)
  return df


In [16]:
shaw=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/shawshank_redemption.csv")
god_father=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/movie3.csv")
matrix=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/matrix.csv")
inter=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/interstellar.csv")
inception=pd.read_csv("https://raw.githubusercontent.com/sanquaorg/Momds_project/main/inception.csv")

In [17]:
inception.rename(columns={"review/movie_title": "review_movie_title","review/username":"review_username","review/rating":"review_rating"}, inplace=True)

In [18]:
shaw=fill("review_movie_title","review_username","review_rating",shaw,"The Shawshank Redemption")
god_father=fill("review_movie_title","review_username","review_rating",god_father,"The Godfather")
matrix=fill("review_movie_title","review_username","review_rating",matrix,"The Matrix")
inter=fill("review_movie_title","review_username","review_rating",inter,"Interstellar")
inception=fill("review_movie_title","review_username","review_rating",inception,"Inception")

In [19]:
def extract(df):
  x=df["review_movie_title"]
  y=df["review_username"]
  z=df["review_rating"]
  df=pd.concat([x,y,z],axis=1)
  return df

In [20]:
shaw=extract(shaw)
god_father=extract(god_father)
matrix=extract(matrix)
inter=extract(inter)
inception=extract(inception)

In [21]:
final = pd.concat([shaw,god_father,matrix,inter,inception], axis=0)
final

,review_movie_title,review_username,review_rating
0,The Shawshank Redemption,hitchcockthelegend,10.0
1,The Shawshank Redemption,Sleepin_Dragon,10.0
2,The Shawshank Redemption,EyeDunno,10.0
3,The Shawshank Redemption,alexkolokotronis,10.0
4,The Shawshank Redemption,kaspen12,10.0
...,...,...,...
8764,Inception,liamgt-82128,9.0
8765,Inception,bobbobwhite,7.0
8766,Inception,Farhad_Aziz,10.0
8767,Inception,gopikirandirector,10.0


In [22]:
final_drop=final.drop_duplicates()
final_drop

,review_movie_title,review_username,review_rating
0,The Shawshank Redemption,hitchcockthelegend,10.0
1,The Shawshank Redemption,Sleepin_Dragon,10.0
2,The Shawshank Redemption,EyeDunno,10.0
3,The Shawshank Redemption,alexkolokotronis,10.0
4,The Shawshank Redemption,kaspen12,10.0
...,...,...,...
8764,Inception,liamgt-82128,9.0
8765,Inception,bobbobwhite,7.0
8766,Inception,Farhad_Aziz,10.0
8767,Inception,gopikirandirector,10.0


In [23]:
username = 'Sleepin_Dragon'

# Filter the DataFrame based on the specified username
filtered_df = final_drop[final_drop['review_username'] == username]
filtered_df

,review_movie_title,review_username,review_rating
1,The Shawshank Redemption,Sleepin_Dragon,10.0
415,The Matrix,Sleepin_Dragon,10.0
135,Inception,Sleepin_Dragon,10.0


In [24]:

pivot_df = pd.pivot_table(final_drop, values='review_rating', index='review_movie_title', columns='review_username', aggfunc='first')
pivot_df


review_username,3xHCCH,851222,AJ4F,AS-5,Aaron1375,AaronCapenBanner,Acello,Achyut_Prashast_Singh,Adam-09265,Adorable,...,zeki-4,zensawa,zetes,zfiany,zhubertX,ziancara,zkonedog,zubsero,zurdulla,zzhhhqing
review_movie_title,,,,,,,,,,,,,,,,,,,,,
Inception,8.0,NaN,5.0,NaN,NaN,7.0,9.0,NaN,NaN,4.0,...,6.0,NaN,10.0,NaN,2.0,10.0,7.0,1.0,9.0,NaN
Interstellar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Godfather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Matrix,NaN,NaN,NaN,NaN,8.0,9.0,NaN,NaN,9.0,NaN,...,NaN,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,10.0
The Shawshank Redemption,NaN,10.0,NaN,10.0,NaN,9.0,NaN,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pivot_df=pivot_df.fillna(0.0)
pivot_df

review_username,3xHCCH,851222,AJ4F,AS-5,Aaron1375,AaronCapenBanner,Acello,Achyut_Prashast_Singh,Adam-09265,Adorable,...,zeki-4,zensawa,zetes,zfiany,zhubertX,ziancara,zkonedog,zubsero,zurdulla,zzhhhqing
review_movie_title,,,,,,,,,,,,,,,,,,,,,
Inception,8.0,0.0,5.0,0.0,0.0,7.0,9.0,0.0,0.0,4.0,...,6.0,0.0,10.0,0.0,2.0,10.0,7.0,1.0,9.0,0.0
Interstellar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Godfather,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Matrix,0.0,0.0,0.0,0.0,8.0,9.0,0.0,0.0,9.0,0.0,...,0.0,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0
The Shawshank Redemption,0.0,10.0,0.0,10.0,0.0,9.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
pivot_df=pivot_df.astype(int)
pivot_df

review_username,3xHCCH,851222,AJ4F,AS-5,Aaron1375,AaronCapenBanner,Acello,Achyut_Prashast_Singh,Adam-09265,Adorable,...,zeki-4,zensawa,zetes,zfiany,zhubertX,ziancara,zkonedog,zubsero,zurdulla,zzhhhqing
review_movie_title,,,,,,,,,,,,,,,,,,,,,
Inception,8,0,5,0,0,7,9,0,0,4,...,6,0,10,0,2,10,7,1,9,0
Interstellar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Godfather,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Matrix,0,0,0,0,8,9,0,0,9,0,...,0,10,0,10,0,0,0,0,0,10
The Shawshank Redemption,0,10,0,10,0,9,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
column_names = {col: str(i) for i, col in enumerate(pivot_df.columns, start=1)}

# Rename the columns of the DataFrame
pivot_df = pivot_df.rename(columns=column_names)
pivot_df

review_username,1,2,3,4,5,6,7,8,9,10,...,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307
review_movie_title,,,,,,,,,,,,,,,,,,,,,
Inception,8,0,5,0,0,7,9,0,0,4,...,6,0,10,0,2,10,7,1,9,0
Interstellar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Godfather,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Matrix,0,0,0,0,8,9,0,0,9,0,...,0,10,0,10,0,0,0,0,0,10
The Shawshank Redemption,0,10,0,10,0,9,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
pivot_df1=pivot_df.copy()
def recommend_movies(user, num_recommended_movies):

  print('The list of the Movies {} Has Watched \n'.format(user))

  for m in pivot_df[pivot_df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_movies = []

  for m in pivot_df[pivot_df[user] == 0].index.tolist():

    index_df = pivot_df.index.tolist().index(m)
    predicted_rating = pivot_df1.iloc[index_df, pivot_df1.columns.tolist().index(user)]
    recommended_movies.append((m, predicted_rating))

  sorted_rm = sorted(recommended_movies, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Movies \n')
  rank = 1
  for recommended_movie in sorted_rm[:num_recommended_movies]:
    
    print('{}: {} - predicted rating:{}'.format(rank, recommended_movie[0], recommended_movie[1]))
    rank = rank + 1

In [29]:
from sklearn.neighbors import NearestNeighbors
def movie_recommender(user, num_neighbors, num_recommendation):
  
  number_neighbors = num_neighbors

  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(pivot_df.values)
  distances, indices = knn.kneighbors(pivot_df.values, n_neighbors=number_neighbors)

  user_index = pivot_df.columns.tolist().index(user)

  for m,t in list(enumerate(pivot_df.index)):
    if pivot_df.iloc[m, user_index] == 0:
      sim_movies = indices[m].tolist()
      movie_distances = distances[m].tolist()
    
      if m in sim_movies:
        id_movie = sim_movies.index(m)
        sim_movies.remove(m)
        movie_distances.pop(id_movie) 

      else:
        sim_movies = sim_movies[:num_neighbors-1]
        movie_distances = movie_distances[:num_neighbors-1]
           
      movie_similarity = [1-x for x in movie_distances]
      movie_similarity_copy = movie_similarity.copy()
      nominator = 0

      for s in range(0, len(movie_similarity)):
        if pivot_df.iloc[sim_movies[s], user_index] == 0:
          if len(movie_similarity_copy) == (number_neighbors - 1):
            movie_similarity_copy.pop(s)
          
          else:
            movie_similarity_copy.pop(s-(len(movie_similarity)-len(movie_similarity_copy)))
            
        else:
          nominator = nominator + movie_similarity[s]*pivot_df.iloc[sim_movies[s],user_index]
          
      if len(movie_similarity_copy) > 0:
        if sum(movie_similarity_copy) > 0:
          predicted_r = nominator/sum(movie_similarity_copy)
        
        else:
          predicted_r = 0

      else:
        predicted_r = 0
        
      pivot_df1.iloc[m,user_index] = predicted_r
  recommend_movies(user,num_recommendation)

In [30]:
movie_recommender("2", 2, 2)

The list of the Movies 2 Has Watched 

The Shawshank Redemption


The list of the Recommended Movies 

1: Inception - predicted rating:10
2: The Godfather - predicted rating:10


5. DGIM

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import deque
import warnings

# Define the DGIM algorithm
def dgim(stream, k):
    buckets = deque(maxlen=k)
    count = 0
    for item in stream:
        if item == '1':
            buckets.append(1)
            count += 1
        else:
            buckets.append(0)
        if len(buckets) > k:
            if buckets[0] == 1:
                count -= 1
            buckets.popleft()
        if len(buckets) == k and buckets.count(1) == 1:
            return count * 2 - k + 1
    return count

# Load the IMDB dataset
df = pd.read_csv('Final.csv')

# Filter out the empty or missing reviews
df = df.dropna(subset=['review_comment'])
df = df[df['review_comment'] != '']

# Preprocess the reviews
stop_words = ['the', 'a', 'an', 'in', 'on', 'at', 'to', 'from', 'of', 'is', 'are', 'was', 'were', 'has', 'have', 'had', 'and', 'or', 'not']
vectorizer = CountVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(df['review_comment'].apply(lambda x: x.lower()))

# Prepare the labeled dataset
y = df['review_rating'].apply(lambda x: '1' if x >= 7 else '0')

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the machine learning model
clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)
with warnings.catch_warnings(record=True) as warn:
    clf.fit(X_train, y_train)

# Evaluate the performance of the model
score = clf.score(X_test, y_test)
print('Accuracy:', score,'\n')

# Predict the sentiment labels of the reviews
predicted_labels = clf.predict(X)
df['sentiment'] = predicted_labels
df['sentiment'] = df['sentiment'].apply(lambda x: '1' if x == '1' else '0')

# Apply DGIM algorithm to count the number of positive and negative reviews for each movie
movies = df['review_movie_title'].unique().tolist()
count_positive_all = 0
count_negative_all = 0
for movie in movies:
    if isinstance(movie, str):  # Check if movie is a string and not NaN
        movie_df = df[df['review_movie_title'] == movie]
        stream = movie_df['sentiment'].to_string(index=False).replace('\n', '')
        k = 10
        count_positive = dgim(stream, k)
        count_negative = dgim(stream[::-1], k)
        print('Movie:', movie)
        print('Number of positive reviews:', count_positive)
        print('Number of negative reviews:', count_negative)
        print()
        count_positive_all += count_positive
        count_negative_all += count_negative

print('Overall positive reviews:', count_positive_all)
print('Overall negative reviews:', count_negative_all)

Accuracy: 0.9593671341244157 

Movie: Inception
Number of positive reviews: 241
Number of negative reviews: 291

Movie: The Shawshank Redemption
Number of positive reviews: 233
Number of negative reviews: 233

Movie: The Godfather
Number of positive reviews: 11
Number of negative reviews: 11

Movie: Interstellar
Number of positive reviews: 1284
Number of negative reviews: 1284

Movie: The Matrix
Number of positive reviews: 11
Number of negative reviews: 11

Overall positive reviews: 1780
Overall negative reviews: 1830
